Description:

Classifies according to business and technical concerns a sample project (a microservice) PlantBasedPizza.LoyaltyPoints.
Classfication results stored as output.csv file in classification/results folder under the name of llm that performed classification.

Key Features:

- LLM Provider Abstraction: Strategy pattern with OpenAI, Anthropic, and Ollama providers
- File Scanner: Scans all 4 LoyaltyPoints projects for .cs and appsettings.json files
- Semantic Classification: Extracts business context, rules, workflows, and integration points
- CSV Output: Structured results for the vector database
- Intermediate Results: JSON files for debugging and comparison
- Provider Comparison: Test all 3 LLM providers on the same code

In [ ]:
from classification.code_scanner import CodeScanner
from pathlib import Path
from classification.classification_pipeline import ClassificationPipeline
from dotenv import load_dotenv
import os


load_dotenv()

LOYALTY_POINTS_APPLICATION = os.getenv("LOYALTY_POINTS_PROJECT_ROOT")
print(LOYALTY_POINTS_APPLICATION)# Update this path

projects = [
    "PlantBasedPizza.LoyaltyPoints.Api.csproj",
    "PlantBasedPizza.LoyaltyPoints.Internal.csproj",
    "PlantBasedPizza.LoyaltyPoints.Shared.csproj",
    "PlantBasedPizza.LoyaltyPoints.Worker.csproj"
]

classification_results = "results/classification"
def create_model_folder(model):
    model_folder = Path(f"{classification_results}/{model}")
    model_folder.mkdir(parents=True, exist_ok=True)

    output_csv = f"{model_folder}/output.csv"
    intermediate_results = f"{model_folder}/intermediate_results"
    return output_csv, intermediate_results

code_scanner = CodeScanner(LOYALTY_POINTS_APPLICATION, projects)

In [ ]:
from llm_providers.anthropic import AnthropicClassifier

anthropic_key = os.environ.get("ANTHROPIC_API_KEY")
output_csv, intermediate_results_dir = create_model_folder("claude3.5")
anth_provider = AnthropicClassifier(api_key=anthropic_key, model="claude-3-5-sonnet-latest")
pipeline = ClassificationPipeline(anth_provider,
                                  scanner=code_scanner,
                                  output_csv=output_csv,
                                  intermediate_dir=intermediate_results_dir)

print("\n=== Running Classification with Anthropic ===")
anthropic_results = pipeline.run_classification()

print("\n=== Classification Complete ===")
print(f"Results saved to: {output_csv}")
print(f"Intermediate results in: {intermediate_results_dir}")

In [ ]:
from llm_providers.ollama import OllamaClassifier

output_csv, intermediate_results_dir = create_model_folder("codellama")
ollama_provider = OllamaClassifier(model="codellama:7b", base_url="http://localhost:11434")
pipeline = ClassificationPipeline(ollama_provider,
                                  scanner=code_scanner,
                                  output_csv=output_csv,
                                  intermediate_dir=intermediate_results_dir)

print("\n=== Running Classification with Ollama ===")
ollama_results = pipeline.run_classification()

print("\n=== Classification Complete ===")
print(f"Results saved to: {output_csv}")
print(f"Intermediate results in: {intermediate_results_dir}")

In [ ]:
from llm_providers.openai import OpenAIClassifier

openai_key = os.getenv("OPENAI_API_KEY")
output_csv, intermediate_results_dir = create_model_folder("gpt4.1")
openai_provider = OpenAIClassifier(api_key=openai_key, model="gpt-4.1-2025-04-14")
pipeline = ClassificationPipeline(openai_provider,
                                  scanner=code_scanner,
                                  output_csv=output_csv,
                                  intermediate_dir=intermediate_results_dir)

print("\n=== Running Classification with OpenAI ===")
openai_results = pipeline.run_classification()

print("\n=== Classification Complete ===")
print(f"Results saved to: {output_csv}")
print(f"Intermediate results in: {intermediate_results_dir}")